In [8]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging
import json

#utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)

import dense_correspondence.evaluation.evaluation
reload(dense_correspondence.evaluation.evaluation)
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

# Object Pursuit
utils.add_object_pursuit_to_python_path()
import dense_correspondence.training.op_training
reload(dense_correspondence.training.op_training)
from dense_correspondence.training.op_training import OPDenseCorrespondenceTraining

cuda = torch.cuda.is_available()
DEVICE = "cuda" if cuda else "cpu"
if cuda:
    print("CUDA GPU!")
else:
    print("CPU!")

setting CUDA_VISIBLE_DEVICES =  0,
CUDA GPU!


## Load the configuration for Object Pursuit

In [2]:
# Load configs used during pretraining
pretrained_path = '/home/ashek/code/data/pdc/trained_models/tutorials/caterpillar_3'
train_config = utils.getDictFromYamlFilename(os.path.join(pretrained_path, 'training.yaml'))
train_config['device'] = DEVICE

In [3]:
# Load data
data_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'object_pursuit.yaml')
data_config = utils.getDictFromYamlFilename(data_config_filename)
data_config['logs_root_path'] = '/home/ashek/code/data/pdc/logs_proto'
dataset = SpartanDataset(config=data_config)

Using SpartanDataset:
   - in train mode
   - number of scenes 38
   - total images:     7232


In [5]:
# Merge custom object pursuit params
def merge_dicts(src, dst):
    # dst is given higher priority
    for k, vsrc in src.items():
        if k not in dst:
            dst[k] = vsrc  # store entire entry, regardless of hierarchy
        elif isinstance(vsrc, dict):
            merge_dicts(src[k], dst[k])
        # else do nothing, dst given priority
        
pursuit_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'object_pursuit.yaml')
pursuit_config = utils.getDictFromYamlFilename(pursuit_config_file)

print("Before:")
print(json.dumps(pursuit_config, indent=2))

print("After:")
merge_dicts(src=train_config, dst=pursuit_config)
print(json.dumps(pursuit_config, indent=2))

Before:
{
  "training": {
    "data_type_probabilities": {
      "SINGLE_OBJECT_ACROSS_SCENE": 0.5, 
      "SINGLE_OBJECT_WITHIN_SCENE": 0.5, 
      "DIFFERENT_OBJECT": 0, 
      "SYNTHETIC_MULTI_OBJECT": 0, 
      "MULTI_OBJECT": 0
    }
  }, 
  "OP": {
    "freeze_backbone": true, 
    "classes": null, 
    "z_dim": 100, 
    "use_backbone": true
  }
}
After:
{
  "device": "cuda", 
  "training": {
    "data_type_probabilities": {
      "SINGLE_OBJECT_ACROSS_SCENE": 0.5, 
      "SINGLE_OBJECT_WITHIN_SCENE": 0.5, 
      "DIFFERENT_OBJECT": 0, 
      "SYNTHETIC_MULTI_OBJECT": 0, 
      "MULTI_OBJECT": 0
    }, 
    "logging_dir": "/home/ashek/code/data/pdc/trained_models/tutorials", 
    "compute_test_loss": false, 
    "domain_randomize": true, 
    "num_matching_attempts": 10000, 
    "weight_decay": 0.0001, 
    "num_non_matches_per_match": 150, 
    "steps_between_learning_rate_decay": 250, 
    "loss_function": "pixelwise_contrastive_loss", 
    "test_loss_num_iterations": 50, 
   

In [9]:
# Run Object Pursuit
train = OPDenseCorrespondenceTraining(dataset=dataset, config=train_config)
train.run()

> /home/ashek/code/dense_correspondence/training/training.py(52)__init__()
     51         ipdb.set_trace()
---> 52         if config is None:
     53             config = DenseCorrespondenceTraining.load_default_config()

ipdb> n
> /home/ashek/code/dense_correspondence/training/training.py(55)__init__()
     54 
---> 55         self._config = config
     56         self.device = config["device"]

ipdb> n
> /home/ashek/code/dense_correspondence/training/training.py(56)__init__()
     55         self._config = config
---> 56         self.device = config["device"]
     57         self._dataset = dataset

ipdb> n
> /home/ashek/code/dense_correspondence/training/training.py(57)__init__()
     56         self.device = config["device"]
---> 57         self._dataset = dataset
     58         self._dataset_test = dataset_test

ipdb> n
> /home/ashek/code/dense_correspondence/training/training.py(58)__init__()
     57         self._dataset = dataset
---> 58         self._dataset_test = dataset_t

KeyError: 'OP'